### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)
school_df.head()
student_df.head()

# Combine the data into a single dataset.  
#school_data_complete = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
#school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [2]:
#Rename the school in both column tables
school_df = school_df.rename(columns={"name":"SchoolName"})
student_df = student_df.rename(columns={"school":"SchoolName"})

In [17]:
#Calculate the total number of schools
number_schools = school_df["school_name"].unique()

In [18]:
#Calculate the total number of students
number_student = student_df["Student ID"].unique()

In [19]:
#Calculate the total budget
total_budget = school_df["budget"].sum()

In [20]:
#Calculate the average math score
average_math_score = student_df["math_score"].mean()

In [21]:
#Calculate the average reading score
average_reading_score = student_df["reading_score"].mean()

In [32]:
#Calculate the percentage of students with a passing math score (70 or greater)
passing_math_score = ((student_df["math_score"] >= 70).sum()/student_df["math_score"].count())*100

In [33]:
#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_score = ((student_df["reading_score"] >= 70).sum()/student_df["reading_score"].count())*100

In [75]:
#Calculate the percentage of students who passed math and reading (% Overall Passing)
passed_math_reading = (student_df.loc[(student_df["math_score"] >= 70) & (student_df["reading_score"] >= 70)].count()[0]/student_df.count()[0])*100

In [76]:
#Create a dataframe to hold the above results
results = pd.DataFrame({"Total Schools":[number_schools.size],
                               "Total Students":[number_student.size],
                               "Budget" : [total_budget],
                               "Average Math Score":[average_math_score],
                               "Average Reading Score":[average_reading_score],
                               "% Passing Math":[passing_math_score],
                               "% Passing Reading": [passing_reading_score],
                               "% Passed_math_reading":[passed_math_reading]})

In [77]:
results = results [["Total Schools", "Total Students", "Budget", "Average Math Score", "Average Reading Score", "% Passing Math",
"% Passing Reading", "% Passed_math_reading"]]
results.head()

,Total Schools,Total Students,Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passed_math_reading
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [175]:
combined_df = student_df.merge(school_df, on='school_name')
groupby_school_df = combined_df.groupby(by="School ID")

In [176]:
#Total Students
total_student = combined_df[['School ID', 'Student ID']].groupby(by='School ID').count()
final_df = school_df.assign(total_student=total_student)

In [177]:
#Per student budget
per_student_budget = final_df['budget'] / final_df['total_student']
final_df = final_df.assign(per_student_budget=per_student_budget)

In [223]:
#Average Math Score
avg_math_score = groupby_school_df.mean()["math_score"].round(2)
final_df = final_df.assign(avg_math_score=avg_math_score)

In [224]:
# Average Reading Score
avg_reading_score = groupby_school_df.mean()["reading_score"].round(2)
final_df = final_df.assign(avg_reading_score=avg_reading_scores)

In [225]:
# Percentage Passing Math
students_passed_df = combined_df.loc[combined_df['math_score'] >= 70]
passed_per_school = (students_passed_df.groupby(by='School ID').count() / combined_df.groupby(by='School ID').count()) * 100
final_df = final_df.assign(percentage_passed_math=passed_per_school[['Student ID']].round(2))

In [226]:
#Percentage Passing Reading
students_passed_df = combined_df.loc[combined_df['reading_score'] >= 70]
passed_per_school = (students_passed_df.groupby(by='School ID').count() / combined_df.groupby(by='School ID').count()) * 100
final_df = final_df.assign(percentage_passed_reading=passed_per_school[['Student ID']].round(2))

In [227]:
#Percentage Overall Passing (The percentage of students that passed math and reading.)
students_passed_df = combined_df.loc[(combined_df['reading_score'] >= 70) & (combined_df['math_score'] >= 70)]
passed_per_school = (students_passed_df.groupby(by='School ID').count() / combined_df.groupby(by='School ID').count()) * 100
final_df = final_df.assign(overall_passed=passed_per_school[['Student ID']].round(2))

In [233]:
school_summary = pd.DataFrame({
    "School": final_df['school_name'],
    "School Type": final_df['type'],
    "Total Students": final_df['total_student'],
    "Per Student Budget": final_df['per_student_budget'],
    "Total School Budget": final_df['budget'],
    "Average Math Score": final_df['avg_math_score'],
    "Average Reading Score": final_df['avg_reading_score'],
    '% Passing Math': final_df['percentage_passed_math'],
    '% Passing Reading': final_df['percentage_passed_reading'],
    "Overall Passing Rate": final_df['overall_passed']
})

school_summary.sort_values(by='School')

,School,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
7,Bailey High School,District,4976,628.0,3124928,77.05,81.03,66.68,81.93,54.64
6,Cabrera High School,Charter,1858,582.0,1081356,83.06,83.98,94.13,97.04,91.33
1,Figueroa High School,District,2949,639.0,1884411,76.71,81.16,65.99,80.74,53.20
13,Ford High School,District,2739,644.0,1763916,77.10,80.75,68.31,79.30,54.29
4,Griffin High School,Charter,1468,625.0,917500,83.35,83.82,93.39,97.14,90.60
3,Hernandez High School,District,4635,652.0,3022020,77.29,80.93,66.75,80.86,53.53
8,Holden High School,Charter,427,581.0,248087,83.80,83.81,92.51,96.25,89.23
0,Huang High School,District,2917,655.0,1910635,76.63,81.18,65.68,81.32,53.51
12,Johnson High School,District,4761,650.0,3094650,77.07,80.97,66.06,81.22,53.54
9,Pena High School,Charter,962,609.0,585858,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [239]:
school_summary.sort_values("Overall Passing Rate", ascending = False).head()

,School,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,582.0,1081356,83.06,83.98,94.13,97.04,91.33
14,Thomas High School,Charter,1635,638.0,1043130,83.42,83.85,93.27,97.31,90.95
4,Griffin High School,Charter,1468,625.0,917500,83.35,83.82,93.39,97.14,90.60
5,Wilson High School,Charter,2283,578.0,1319574,83.27,83.99,93.87,96.54,90.58
9,Pena High School,Charter,962,609.0,585858,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [240]:
school_summary.sort_values("Overall Passing Rate", ascending = True).head()

,School,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
11,Rodriguez High School,District,3999,637.0,2547363,76.84,80.74,66.37,80.22,52.99
1,Figueroa High School,District,2949,639.0,1884411,76.71,81.16,65.99,80.74,53.20
0,Huang High School,District,2917,655.0,1910635,76.63,81.18,65.68,81.32,53.51
3,Hernandez High School,District,4635,652.0,3022020,77.29,80.93,66.75,80.86,53.53
12,Johnson High School,District,4761,650.0,3094650,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [245]:
ninth_math = combined_df.loc[combined_df['grade'] == '9th'].groupby('school_name')["math_score"].mean().round(2)
tenth_math = combined_df.loc[combined_df['grade'] == '10th'].groupby('school_name')["math_score"].mean().round(2)
eleventh_math = combined_df.loc[combined_df['grade'] == '11th'].groupby('school_name')["math_score"].mean().round(2)
twelfth_math = combined_df.loc[combined_df['grade'] == '12th'].groupby('school_name')["math_score"].mean().round(2)

pd.DataFrame({
    "9th": ninth_math,
    "10th": tenth_math,
    "11th": eleventh_math,
    "12th": twelfth_math
})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [246]:
ninth_math = combined_df.loc[combined_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean().round(2)
tenth_math = combined_df.loc[combined_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean().round(2)
eleventh_math = combined_df.loc[combined_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean().round(2)
twelfth_math = combined_df.loc[combined_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean().round(2)

pd.DataFrame({
    "9th": ninth_math,
    "10th": tenth_math,
    "11th": eleventh_math,
    "12th": twelfth_math
})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [251]:
bins = [0, 584, 629, 644, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]
scoreByBudget = school_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].groupby(pd.cut(school_summary["Per Student Budget"], bins=bins, labels=group_names )).mean().round(2)
scoreByBudget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
0 to 585,83.45,83.94,93.46,96.61,90.37
585 to 615,81.90,83.16,87.13,92.72,81.42
615 to 645,78.52,81.62,73.48,84.39,62.86
645 to 675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [253]:
bins = [0, 1000, 2000, 60000]
group_names = ["Small < 1000", "Medium 1000 to 2000", "Large 2000 to 60000"]
scoreBySize = school_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].groupby(pd.cut(school_summary["Total Students"], bins=bins, labels=group_names)).mean()
scoreBySize.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small < 1000,83.820,83.92500,93.55000,96.10000,89.885
Medium 1000 to 2000,83.374,83.86800,93.59800,96.79000,90.620
Large 2000 to 60000,77.745,81.34375,69.96375,82.76625,58.285


## Scores by School Type

* Perform the same operations as above, based on school type

In [301]:
group_names = {"Charter": 0, "District": 1}
bins = [-1] + list(group_names.values())
school_summary = school_summary.assign(school_type_index=school_summary['School Type'].apply(lambda x: group_names[x]))
scoreByType = school_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]].groupby(pd.cut(school_summary["school_type_index"], bins=bins, labels=list(group_names.keys()))).mean()
scoreByType.head()




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_type_index,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,90.431250
District,76.955714,80.965714,66.548571,80.798571,53.671429


In [ ]:
#Trends:
    
    #The trends show that the lower budget schools have higher grades in Maths
    #The second trend can also point out the clear difference between the Overall Passing rate of the students from District.